In [ ]:
import ngsolve as ng
from numpy import pi
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry
from fiberamp.fiber import ModeSolver

In [ ]:
class SquareFiber(ModeSolver):
    
    def __init__(self, Rsqr=.5, R=1.5, Rout=2.5, ref=0, curve=3):
        
        self.Rsqr = Rsqr
        self.R = R
        self.Rout = Rout
        
        geo = SplineGeometry()
        geo.AddRectangle((-Rsqr,-Rsqr), ((Rsqr,Rsqr)), leftdomain=1, rightdomain=2)
        geo.AddCircle((0,0), r=self.R, leftdomain=2, rightdomain=3)
        geo.AddCircle((0,0), r=self.Rout, leftdomain=3, bc='OuterCircle')
        
        geo.SetMaterial(1, 'core')
        geo.SetMaterial(2, 'clad')
        geo.SetMaterial(3, 'Outer')
        
        geo.SetDomainMaxH(1, .2)
        geo.SetDomainMaxH(2, .5)
        geo.SetDomainMaxH(3, 2)
        
        self.geo = geo
        self.mesh = ng.Mesh(geo.GenerateMesh())
        for r in range(ref):
            self.mesh.ngmesh.Refine()
        self.mesh.ngmesh.SetGeometry(self.geo)
        self.mesh.Curve(curve)
        
        self.n0 = 1.45
        self.n_core = 1.48
        self.n_clad = 1.45
        self.wavelength = .5e-6
        self.k = 2*pi/self.wavelength
        self.L = 1.2e-6

        self.N = ng.CoefficientFunction([self.n_core, self.n_clad, self.n0])
        self.V = (self.k * self.L)**2 * (self.n0**2 - self.N**2)
        self.index = self.N

        super().__init__(self.mesh, self.L, self.n0)

In [ ]:
Rsqr_in = .5
Rsqr_out = 1

R = 2
Rout = 3

geo = SplineGeometry()
geo.AddRectangle((-Rsqr_in, -Rsqr_in), ((Rsqr_in, Rsqr_in)),
                 leftdomain=1, rightdomain=2)
geo.AddRectangle((-Rsqr_out, -Rsqr_out), ((Rsqr_out, Rsqr_out)),
                 leftdomain=2, rightdomain=3)
geo.AddCircle((0, 0), r=R, leftdomain=3, rightdomain=4)
geo.AddCircle((0, 0), r=Rout, leftdomain=4, bc='OuterCircle')

geo.SetMaterial(1, 'core')
geo.SetMaterial(2, 'square_clad')
geo.SetMaterial(3, 'outer_clad')
geo.SetMaterial(4, 'Outer')

geo.SetDomainMaxH(1, .2)
geo.SetDomainMaxH(2, .5)
geo.SetDomainMaxH(3, 1)
geo.SetDomainMaxH(4, 2)

mesh = ng.Mesh(geo.GenerateMesh())

In [ ]:
Draw(mesh)

In [ ]:
A = SquareFiber(ref=1, curve=10)

In [ ]:
Draw(A.V, A.mesh)

In [ ]:
a = (A.sqrZfrom(A.k * A.n_core)**.5).imag

In [ ]:
z, y, _, _, _, _ = A.leakymode(5, ctr=3.3j, rad = .1, alpha=10, npts=4, nspan=2, niterations=10,nrestarts=0 )

In [ ]:
for f in y:
    Draw(1e2*f, A.mesh)

In [ ]:
a = A.sqrZfrom(A.k * A.n_core)

In [ ]:
a

In [ ]:
betas, zs, E, phi, _ = A.leakyvecmodes(ctr=-10.889, rad=.1, p=3, npts=4, nspan=7, 
                                       niterations=20, nrestarts=0, stop_tol=1e-9)

In [ ]:
for f in E:
    Draw(f.real, A.mesh)

In [ ]:
for p in phi:
    Draw(1e2*p, A.mesh)